In [1]:
%load_ext autoreload
%autoreload 2

import os
from tqdm import tqdm

from processingmm import batch_processing
from freezingeffect import selection_of_ROIs, automatic_ROI_propagation, collect_data_propagated
from freezingeffect.helpers import load_parameters_ROIs

## 1. Process the Mueller Matrices of the folders

In [2]:
# set the parameters to run the script
data_folder_path = r'C:\Users\romai\Desktop\Freezing_dev'
# data_folder_path = r'C:\Users\romai\Documents\illustrators\pathology_protocol\imgs\effect_freezing'
directories = [data_folder_path]
calib_directory = r'C:\Users\romai\Desktop\calib'

# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
# set the parameters to be used for the line visualisation with the parameter parameter_set
batch_processing.batch_process(directories, calib_directory, run_all = False, parameter_set = 'PD')

0it [00:00, ?it/s]


[]

## 2. Specify the folders that will be processed and clean them (remove precedent processing results)

In [3]:
path_folder_temp, path_align_folder = selection_of_ROIs.create_alignment_folder()
base_dirs = selection_of_ROIs.get_the_base_dirs(data_folder_path)
selection_of_ROIs.clean_the_alignment_folders(path_align_folder)
selection_of_ROIs.create_the_masks(data_folder_path)

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:38<00:00,  2.14s/it]


## 3. Loop over all the folders that will be processed

#### 3.1. loop over all the folders that needs to be processed

In [4]:
import time

# tissue_types = ['WM', 'GM']
tissue_types = ['WM', 'GM']
for path_folder in tqdm(base_dirs):
    propagation_lists = {}
    
    for tissue_type in tissue_types:
        # 1. set up the parameters and load the Mueller Matrix
        WM = tissue_type == 'WM'
        
        if WM:
            new_folder_name = 'WM_1'
        else:
            new_folder_name = 'GM_1'
        propagated = False
        
        # 2. automatic selection and preparation for the propagation of the ROIs
        propagation_lists[tissue_type] = selection_of_ROIs.square_selection(path_folder_temp, path_folder, WM, 
                                                                            tissue_type, path_align_folder)
    # 3. actually do the alignment
    propagation_lists = automatic_ROI_propagation.generate_combined_mask(propagation_lists, path_align_folder)
    automatic_ROI_propagation.do_alignment(path_align_folder)
    output_folders = automatic_ROI_propagation.move_computed_folders(path_align_folder)
    
    # 4. and collect the data after propagation
    for tissue_type in tissue_types:
        WM = tissue_type == 'WM'
        collect_data_propagated.collect_data_propagated(WM, path_align_folder, propagation_lists[tissue_type], 
                                                            output_folders, data_folder_path)

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

89.65166610564495 29.769470662895962
153.97721572384 53.20238730836353
89.88886979256746 50.5761891653328
102.30401148346884 30.044554679362772
128.90268991173951 46.38462991226296
65.6091439884885 43.94278517309993
95.73043154541702 42.23219389380778
139.60094076253648 36.26721964182922
83.98530923393665 42.22526244954873
105.24393650291138 41.6850379844772
78.99555511048574 28.06126130460058
63.79988952406981 53.47584204184021
145.78974029286684 25.40436263758444
105.51474983653766 51.94110661252707
136.6775481987562 18.57760143539888
125.20396992972596 34.09100459257567
95.67073935351856 27.65836560082873
171.45028212982862 60.32806477301263
110.21691522691239 23.819947654080558
105.71891884673384 20.323136010965754
143.04117875348354 33.60399703648989
86.71587470425126 27.72451902479831
48.74194797593483 30.019288838319483
88.60987666680481 25.321127859967557
123.51538705400505 34.00963252461094
23.571145064107753 18.6391648160678
92.12416709936765 12.652183042104431
101.0816772091

 17%|█████████████▊                                                                     | 1/6 [02:32<12:42, 152.45s/it]

81.51439011379311 27.46229070739487
11.33148066449439 18.527209822376843
176.7085179699124 20.92186168391878
62.60196527318006 40.80543753644444
12.148490737121067 44.96956782149168
63.7278740948927 38.21162277671202
136.31452314645733 22.20775179507353
148.53740412072844 20.364794375818708
110.91259881070935 21.029132010076456
14.728354797825977 23.642370789790395
9.688502263099046 30.01652057446085
107.89734353057585 19.28021379946037
78.79084379890425 41.87059372815051
160.1967025596744 17.949045292432526
Folder already exists, trying again...
8.90265934038685 31.727254771461162
43.10181076089478 35.545408301421844
118.15026614142477 25.525902044860032
3.530289700896222 14.585274432739293
0.7386511738441105 26.877996966975555
3.954573689051509 31.29652336766801
39.284908882557914 27.996078126891522
31.164070629995614 34.136844194684905
139.34068590881026 38.77836450860989
12.446411215288746 21.800896867693304
80.15855508639325 17.43392007433593
66.1385224492516 26.187728296626435
14

 33%|███████████████████████████▋                                                       | 2/6 [05:03<10:05, 151.40s/it]

...

 67%|██████████████████████████████████████████████████████▋                           | 4/6 [50:54<37:16, 1118.39s/it]

101.54154924549287 28.742547034676164
40.5792247182511 22.91713132267928
62.231891504611625 27.89527853708883
45.62173112569346 28.46435660706868
43.05537060345968 21.794885224623247
169.23570763937087 8.180034002734313
167.21102608139176 9.563308590855998
26.781525927638256 69.71891391767706
38.19237414974108 30.93158274548469
165.75115531046922 33.42957749679319
98.42223332895634 20.772462708224154
20.283382990306862 33.24724609947528
101.01121791406818 50.227687028469205
107.37545986102138 12.991251878773525
169.42451626554214 17.412263483091305
45.156964497213735 27.234324025665387
12.928898105857565 48.13484248434329
57.433436797823276 13.005227641539426
63.08683675745913 44.194908429076996
100.21508855583205 16.656029315639007
27.46834607719808 29.883849218065365
58.061216500634785 43.662443375824004
90.61312985533311 10.709488402734642
77.7864543008032 37.800850219836676
173.014981989078 45.502605412902554
141.26958471067647 6.940058167931206
150.28079431887397 7.523257881738889

 83%|█████████████████████████████████████████████████████████████████████▏             | 5/6 [54:05<13:03, 783.80s/it]

172.0219142928075 17.315229263457244
156.23046719777273 37.365481978637895
44.592209338766935 26.793282795824826
157.27172363170112 30.05455947983888
144.0819313463188 42.67829981069262
41.39807043171738 49.07390557218384
52.72964734236954 40.67833295535
34.35934576703566 20.279848227919107
45.66385888446644 24.493730365383477
169.70925051864273 12.036947174252033
155.01968696777973 49.89798058732772
57.77917255173097 38.76781360692734
136.58977863104872 52.77254140222558
149.720003303662 26.477238601132544
41.643533991318776 32.905057781883094
62.655801211633964 14.103095150663483
163.94959162383157 22.040659165497182
158.11223747221342 21.428161772325385
18.99777282322092 34.3885942228377
76.73048046087264 36.53616763904895
11.355426709590516 35.29348925290047
177.4894651635297 38.89310499847611
45.03619118491434 24.390340923428106
36.43631511411215 16.484388127011858
87.9494399893417 23.44314594376095
124.34567673875416 18.73222605483699
160.06544740305856 34.70025054602369
96.57054

100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [57:03<00:00, 570.63s/it]


## 4. Compare the parameters

#### 4.1. Get the folders for the measurements of interest

In [31]:
from freezingeffect import parameter_comparaison

In [32]:
# get the parameters for the comparaison
path_folder = data_folder_path
wavelength, match_sequence, measurement, max_, max_nb_of_squares = parameter_comparaison.get_parameters()
max_nb_of_squares = max_nb_of_squares * len(base_dirs)


# and the folders containing the data
data_folders, base_folders, to_be_removed = parameter_comparaison.get_data_folders(path_folder, wavelength, match_sequence)

data_folders[0:5] + data_folders[-5:]

['C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_1',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_10',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_11',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_12',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_13',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S6_1\\polarimetry\\550nm\\50x50_images\\WM_5',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S6_1\\polarimetry\\550nm\\50x50_images\\WM_6',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S6_1\\polarimetry\\550nm\\50x50_images\\WM_7',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR

#### 4.2. Get the data for the measurement

In [33]:
data_types, data_fixation = parameter_comparaison.get_data(data_folders)

data_fixation['GM'][list(data_fixation['GM'].keys())[0]]

,fname,parameter,mean,stdev,max,median,square_size
0,2023-04-06_T_FRE-FR-T0_FR_S1_1,azimuth,23.571145,18.639165,28.314607,23.571145,None
1,2023-04-06_T_FRE-FR-T0_FR_S1_1,depolarization,0.766462,0.059562,0.737975,0.764285,None
2,2023-04-06_T_FRE-FR-T0_FR_S1_1,diattenuation,0.048532,0.011463,0.040816,0.048068,None
3,2023-04-06_T_FRE-FR-T0_FR_S1_1,retardance,15.346636,6.263379,13.670886,15.207584,None
4,2023-04-11_T_FRE-FR-T1_FR_S1_1,azimuth,91.231137,17.568209,80.898876,91.231137,None
5,2023-04-11_T_FRE-FR-T1_FR_S1_1,depolarization,0.742700,0.044027,0.710127,0.740853,None
6,2023-04-11_T_FRE-FR-T1_FR_S1_1,diattenuation,0.050102,0.012571,0.040816,0.049425,None
7,2023-04-11_T_FRE-FR-T1_FR_S1_1,retardance,8.333395,3.479396,6.835443,8.004341,None
8,2023-04-12_T_FRE-FR-T2_FX_S1_1,azimuth,84.749739,22.491814,72.808989,84.749739,None
9,2023-04-12_T_FRE-FR-T2_FX_S1_1,depolarization,0.785589,0.028613,0.779747,0.784519,None


In [34]:
# remove the ROI moving from white matter to grey matter and vice versa
for data_type, dfed in data_fixation.items():
    for folder, df in dfed.items():
        to_remove_dfed = to_be_removed[folder.split('\\polarimetry')[0]]
        for val in to_remove_dfed:
            for fol in val:
                if fol[0] == folder.split('\\')[-1].split('_align')[0]:
                    df = df.apply(parameter_comparaison.find_match_seq, match_seq = fol[1], axis = 1)
                    data_fixation[data_type][folder] = df

In [35]:
data_fixation['GM'][list(data_fixation['GM'].keys())[0]]

,fname,parameter,mean,stdev,max,median,square_size
0,2023-04-06_T_FRE-FR-T0_FR_S1_1,azimuth,23.571145,18.639165,28.314607,23.571145,None
1,2023-04-06_T_FRE-FR-T0_FR_S1_1,depolarization,0.766462,0.059562,0.737975,0.764285,None
2,2023-04-06_T_FRE-FR-T0_FR_S1_1,diattenuation,0.048532,0.011463,0.040816,0.048068,None
3,2023-04-06_T_FRE-FR-T0_FR_S1_1,retardance,15.346636,6.263379,13.670886,15.207584,None
4,2023-04-11_T_FRE-FR-T1_FR_S1_1,azimuth,91.231137,17.568209,80.898876,91.231137,None
5,2023-04-11_T_FRE-FR-T1_FR_S1_1,depolarization,0.742700,0.044027,0.710127,0.740853,None
6,2023-04-11_T_FRE-FR-T1_FR_S1_1,diattenuation,0.050102,0.012571,0.040816,0.049425,None
7,2023-04-11_T_FRE-FR-T1_FR_S1_1,retardance,8.333395,3.479396,6.835443,8.004341,None
8,2023-04-12_T_FRE-FR-T2_FX_S1_1,azimuth,84.749739,22.491814,72.808989,84.749739,None
9,2023-04-12_T_FRE-FR-T2_FX_S1_1,depolarization,0.785589,0.028613,0.779747,0.784519,None


In [36]:
#### Set up the parameters to be studied
parameters = ['retardance', 'depolarization', 'diattenuation', 'azimuth']
metrics = ['median', 'mean', 'max']
times = ['T0', 'T1', 'T2']
    
# create the directories that will be used to store the output of the analysis
folder = parameter_comparaison.create_output_directories(measurement, data_types, parameters = parameters)

In [37]:
### Get the data and save it into xlsx file
to_combine = parameter_comparaison.get_and_organize_data(data_fixation, metrics, data_types, times, folder, max_,
                          parameters = ['retardance', 'depolarization', 'diattenuation', 'azimuth'])

to_combine[('retardance', 'WM', 'median')]

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,7.457162,12.637980,14.139963,12.316119,36.479351,14.442026,24.658115,15.877166,8.728315,14.221657,...,19.280461,22.682055,22.992927,16.841642,17.708610,15.138147,12.783854,14.629589,19.726193,18.218592
1,16.791370,11.521227,13.259835,13.852658,20.227749,15.877485,15.531103,18.508755,14.056647,13.206275,...,15.962553,14.256542,14.972237,15.645076,19.896065,11.442979,14.140248,13.702106,16.052381,15.448424
2,17.062550,15.826493,25.462917,13.125743,17.601635,12.410693,23.778649,31.317123,21.576054,12.907298,...,13.611643,10.371843,13.323978,12.008196,15.974230,11.964259,14.786276,17.097111,11.986680,12.155298


In [39]:
to_combine_cv = {}
for idx, val in to_combine.items():
    if idx[0] == 'azimuth':
        to_combine_cv[idx] = parameter_comparaison.combine_data_cv(val, len(times), azimuth = True)
    else:
        to_combine_cv[idx] = parameter_comparaison.combine_data_cv(val, len(times))
        
to_combine_cv[('retardance', 'WM', 'median')]

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,0.444107,1.096930,1.066375,0.889080,1.803431,0.909592,1.587660,0.857819,0.620939,1.076886,...,1.207856,1.590993,1.535704,1.076482,0.890056,1.322920,0.904076,1.067689,1.228864,1.179317
1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,1.016150,1.373681,1.920304,0.947525,0.870173,0.781654,1.531034,1.692017,1.534936,0.977361,...,0.852723,0.727515,0.889912,0.767538,0.802884,1.045555,1.045687,1.247773,0.746723,0.786831


#### 4.3. Statistical analysis

In [40]:
#### Get the data to be used for the comparisons
# recombine the data in a different format
data_all_recombined = parameter_comparaison.recombine_data_tests(to_combine, times)
data_all_recombined_cv = parameter_comparaison.recombine_data_tests(to_combine_cv, times)

In [41]:
#### Comparaison of the means and the means of the fold change
# perform the statistical test for both fold change and mean values
paired_test = parameter_comparaison.t_test(data_all_recombined, times, 'median')# times[1:]])
paired_test_cv = parameter_comparaison.t_test(data_all_recombined_cv, times, 'median')# times[1:]])
df_grouped = parameter_comparaison.create_test_df(paired_test, parameter = 'median')
df_grouped_cv = parameter_comparaison.create_test_df(paired_test_cv, parameter = 'median')
df_grouped_cv

Z       p_value    n_1    n_2  before/GM  \
parameter      data_type time                                                   
azimuth        GM        T0        0.0  1.876723e-49  128.0  128.0        0.0   
                         T1     8192.0  1.000000e+00  128.0  128.0        0.0   
                         T2        0.0  3.996383e-48  128.0  116.0        0.0   
               WM        T0        0.0  1.914093e-52  136.0  136.0        0.0   
                         T1     9248.0  1.000000e+00  136.0  136.0        0.0   
                         T2        0.0  4.034459e-52  136.0  133.0        0.0   
depolarization GM        T0     9600.0  1.108092e-02  128.0  128.0        1.0   
                         T1     8192.0  1.000000e+00  128.0  128.0        1.0   
                         T2      640.0  1.787749e-40  128.0  116.0        1.0   
               WM        T0    11968.0  7.401002e-06  136.0  136.0        1.0   
                         T1     9248.0  1.000000e+00  136.0  136.0        1.0   
                         T2    14688.0  2.542631e-21  136.0  133.0        1.0   
retardance     GM        T0     4864.0  1.911087e-09  128.0  128.0        1.0   
                         T1     8192.0  1.000000e+00  128.0  128.0        1.0   
                         T2    11520.0  8.877957e-16  128.0  116.0        1.0   
               WM        T0     7752.0  1.371552e-02  136.0  136.0        1.0   
                         T1     9248.0  1.000000e+00  136.0  136.0        1.0   
                         T2    10472.0  1.648807e-02  136.0  133.0        1.0   

                               before/GM std   after/WM  after/WM std  
parameter      data_type time                                          
azimuth        GM        T0              0.0  26.005689     23.218996  
                         T1              0.0   0.000000      0.000000  
                         T2              0.0  19.057235     19.384433  
               WM        T0              0.0  25.258100     21.141495  
                         T1              0.0   0.000000      0.000000  
                         T2              0.0  17.995813     19.211723  
depolarization GM        T0              0.0   0.990021      0.038996  
                         T1              0.0   1.000000      0.000000  
                         T2              0.0   1.071343      0.039382  
               WM        T0              0.0   0.996223      0.012499  
                         T1              0.0   1.000000      0.000000  
                         T2              0.0   0.993709      0.011103  
retardance     GM        T0              0.0   1.500521      0.774655  
                         T1              0.0   1.000000      0.000000  
                         T2              0.0   0.851294      0.265853  
               WM        T0              0.0   1.128884      0.376529  
                         T1              0.0   1.000000      0.000000  
                         T2              0.0   0.975502      0.218128

In [42]:
# finally, save the results
path_statistical_test = os.path.join('results', measurement)
df_grouped.to_excel(os.path.join(path_statistical_test, 'statistical_test.xlsx'))
df_grouped_cv.to_excel(os.path.join(path_statistical_test, 'statistical_test_cv.xlsx'))

#### 4.4. Combine the results in a single excel file

In [43]:
# this cell allows to combine the data obtained for both grey and white matter in a table that can then be used
# to plot the results using Graphpad Prism
retardance = {}
depolarization = {}
for key, param in to_combine.items():
    if key[0] == 'retardance' and key[2] == 'median':
        retardance[key[1]] = param
    elif key[0] == 'depolarization' and key[2] == 'median':
        depolarization[key[1]] = param
        
retardance_cv = {}
depolarization_cv = {}
azimuth_cv = {}
for key, param in to_combine_cv.items():
    if key[0] == 'retardance' and key[2] == 'median':
        retardance_cv[key[1]] = param
    elif key[0] == 'depolarization' and key[2] == 'median':
        depolarization_cv[key[1]] = param
    else:
        if key[2] == 'mean':
            azimuth_cv[key[1]] = param
        
retardance = parameter_comparaison.create_df_prism(retardance, max_nb_of_squares)
depolarization = parameter_comparaison.create_df_prism(depolarization, max_nb_of_squares)
    
retardance_cv = parameter_comparaison.create_df_prism(retardance_cv, max_nb_of_squares)
depolarization_cv = parameter_comparaison.create_df_prism(depolarization_cv, max_nb_of_squares)
azimuth_cv = parameter_comparaison.create_df_prism(azimuth_cv, max_nb_of_squares)

retardance_cv

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,1.899917,1.578861,1.711057,NaN,NaN,1.698262,NaN,1.071818,0.819436,1.973394,...,1.207856,1.590993,1.535704,1.076482,0.890056,1.322920,0.904076,1.067689,1.228864,1.179317
1,1.000000,1.000000,1.000000,NaN,NaN,1.000000,NaN,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,0.794690,0.595933,1.144890,NaN,NaN,1.526351,NaN,1.003291,0.781505,0.762137,...,0.852723,0.727515,0.889912,0.767538,0.802884,1.045555,1.045687,1.247773,0.746723,0.786831


In [44]:
# finally save all the tables
path_data_prism = os.path.join('results', measurement)
retardance.to_excel(os.path.join(path_data_prism, "retardance_prism.xlsx"))
depolarization.to_excel(os.path.join(path_data_prism, "depolarization_prism.xlsx"))
retardance_cv.to_excel(os.path.join(path_data_prism, "retardance_prism_cv.xlsx"))
depolarization_cv.to_excel(os.path.join(path_data_prism, "depolarization_prism_cv.xlsx"))
azimuth_cv.to_excel(os.path.join(path_data_prism, "azimuth_prism_cv.xlsx"))